In [2]:
%load_ext autoreload
%autoreload 2

# Imports
import re
import pandas as pd
import numpy as np
from cellstructure import Datasheet
from files import get_list_of_files_with_ext, join
import yaml

In [266]:
# path_to_pdf = "/Volumes/T7/thesis-data/test/selected_excel/5a1e4d632b8c5.xlsx"
path_to_pdf = "/Volumes/T7/thesis-data/test/single_excel/test.xlsx"

ds = Datasheet()
ds.load_tables_from_excel(path_to_excel=path_to_pdf)

print(ds.name)

table = ds.tables[0].raw_df

print(table)

test.xlsx
                         0              1        2        3        4       5  \
0   Electrical Performance                                                     
1                      No.  Efficiency(%)  Pmpp(W)  Umpp(V)  Impp(A)  Uoc(v)   
2                       01         22.400    5.640    0.591    9.550   0.689   
3                       02         22.300    5.620    0.590    9.520   0.688   
4                       03         22.200    5.590    0.589    9.500   0.687   
5                       04         22.100    5.570    0.588    9.470   0.686   
6                       05         22.000    5.540    0.587    9.440   0.685   
7                       06         21.900    5.520    0.586    9.420   0.684   
8                       07         21.800    5.490    0.585    9.390   0.683   
9                       08         21.700    5.470    0.584    9.360   0.682   
10                      09         21.600    5.440    0.583    9.340   0.681   
11                      10    

In [264]:

def is_table_vertical(raw_df: pd.DataFrame):
    """
    This function will determine whether the table of values specified
    is vertical or horizontal. This is important in order to correctly
    extract out the values and assign them a label.

    Parameters:
        raw_df:
            This is the raw pandas dataframe that contains the values
            of the table that is extracted from the PDF files.
    
    Returns:
        result:
            This is the boolean result that is True if the table is
            vertical otherwise False.
    """

    # Convert the DataFrame to a 2D list so make the iterations more
    # Pythonic

    table_as_list = table.values.tolist()

    # Create an empty list that will contain the raw values on which
    # the decision of either vertical or horizontal will be made.
    table_of_extracted_values = []

    # Iterate over the rows in the table
    for row in table_as_list:
        
        # This empty list will hold all the values that are extracted
        # from the current row
        extracted_row = []

        # Iterate over all columns in the current row
        for col in row:
            
            match = re.search(
                pattern="\d+.\d+|\d+",
                string=str(col)
            )

            if match is not None:

                col = match.group(0)

                # The following try-except clauses are used to determine if
                # the current element is a number or not.
                try:
                    curr_val = float(col)
                    extracted_row.append(curr_val)

                except ValueError as e:
                    pass
            
        # If something is extracted then add it to the main matrix
        if len(extracted_row) != 0:
            table_of_extracted_values.append(extracted_row)

    # If no values were extracted then return nothing
    if not table_of_extracted_values:
        return None

    # Determine if the extracted list of lists is a valid matrix, to do
    # that the lengths of all the rows are computed. If all the lengths
    # of the rows are equal then we have valid matrix otherwise not.
    valid_matrix = all([len(x)==len(table_of_extracted_values[0]) for x in table_of_extracted_values])

    # Exit with None if the matrix is not valid.
    if not valid_matrix:
        return None

    table_of_extracted_values = np.asarray(table_of_extracted_values)

    # print(table_of_extracted_values)

    # Calculate the variance for the extracted matrix in both the horizontal
    # and the vertical directions. The direction of least variation will be
    # the direction of the table
    vertical_var = np.sum(np.var(table_of_extracted_values, axis=1))
    horizontal_var = np.sum(np.var(table_of_extracted_values, axis=0))

    if horizontal_var > vertical_var:
        return False
    else:
        return True




In [265]:
result = is_table_vertical(table)

if result is None:
    print("Error Determining Axis")

elif result:
    print("Vertical Table")

else:
    print("Horizontal Table")

Error Determining Axis


In [258]:
# Implementation for vertical table

pattern_efficiency = "eff((?!code).)*$|ncell|model\(%\)"
pattern_isc = "isc|shortcircuit(current)?"
pattern_voc = "[uv]oc|opencircuit(voltage)?|vm\W|circuitvoltage"
pattern_impp = "^imp+|^ip+m|(max\.?(imum)?)?powercurrent|currentat\s(max\.?(imum)?)?power|im(?!um)$"
pattern_vmpp = "^[uv]mp+|^[uv]p+m|(max\.?(imum)?)?powervoltage|voltageat\s(max\.?(imum)?)?power"
pattern_pmpp = "pmax|pmpp|ppm|^pm|^power$|[\(\[]wp*[\)\]]|(average|rated|charged)power"
pattern_ff = "^ff|fillfactor"

pattern_eff_vals = "/\d+\.\d+%(\s|)-(\s|)\d+\.\d+%|\d+%(\s|)-(\s|)\d+%|\d+%|\d+\.\d+%|\d+\.\d+|\d+/gm"
pattern_other_vals = "\d+.\d+\D|\d+\D|\d+"

def check_col_for_pattern(
    pattern: str,
    col: int,
    table: pd.DataFrame
    ) -> bool:
    """
    This function checks the provided column in the provided table
    for the presence of the regex pattern.

    Parameters:
        pattern:
            This is the regex pattern to check

        col:
            This is the column in the table to check

        table:
            This is the table in which the column to check exists

    Returns:
        result:
            A boolean type result that is True if the column contains the requireed
            pattern otherwise False.
    """


    # Check each row of the column if the pattern exists
    row_check = table.iloc[:, col].str.contains(
        pattern,
        flags=re.IGNORECASE,
        regex=True,
        na=False
        )

    # Return True if the pattern exists in any of the rows or False
    # if it doesnt.
    return any(row_check)

def extract_values_from_column(
    table: pd.DataFrame,
    cols: list,
    pattern: str
    ) -> list:
    """
    This function will iterate over the provided column in the provided table
    and extract the values using the provided pattern and put them in a list.

    Parameters:
        table:
            This is the table which contains the column from which to extract
            the values from.
        col_no:
            This is the list of columns that need to be searched.
        pattern:
            This is the regex pattern that will be used to match the values in
            order to extract them.

    Returns:
        values:
            This is the list of all the values extracted.
    """

    vals = []

    for col in cols:

        raw = table.iloc[:, col].values.tolist()

        for element in raw:
            result = re.search(
                pattern=pattern,
                string=str(element)
            )

            if result is not None:
                vals.append(element)
    
    return vals

def get_electrical(
    table: pd.DataFrame,
    axis: str
    ) -> None:
    """
    This function will attempt to extract the electrical characteristics from the
    table provided.

    Paramters:
        table:
            A table that is known to contain the electrical characteristics and contains
            values along the vertical axis.
        axis:
            This is the axis along which the values exist in the table. The possible choices
            are either "vertical" or "horizontal"
    """

    # Prepare the column number lists
    eff_cols = []
    isc_cols = []
    voc_cols = []
    impp_cols = []
    vmpp_cols = []
    pmpp_cols = []
    ff_cols = []


    n_cols = table.shape[1]
    print("Number of Columns: " + str(n_cols))

    # Locate the positions of required columns
    for i in range(0, n_cols):
        if check_col_for_pattern(pattern_efficiency, i, table):
            eff_cols.append(i)
        elif check_col_for_pattern(pattern_isc, i, table):
            isc_cols.append(i)
        elif check_col_for_pattern(pattern_voc, i, table):
            voc_cols.append(i)
        elif check_col_for_pattern(pattern_impp, i, table):
            impp_cols.append(i)
        elif check_col_for_pattern(pattern_vmpp, i, table):
            vmpp_cols.append(i)
        elif check_col_for_pattern(pattern_pmpp, i, table):
            pmpp_cols.append(i)
        elif check_col_for_pattern(pattern_ff, i, table):
            ff_cols.append(i)

    # print(table)

    # Efficiency values extraction
    eff_vals = extract_values_from_column(
        table=table,
        cols=eff_cols,
        pattern=pattern_eff_vals
    )

    # Isc values extraction
    isc_vals = extract_values_from_column(
        table=table,
        cols=isc_cols,
        pattern=pattern_other_vals
    )

    # Voc values extraction
    voc_vals = extract_values_from_column(
        table=table,
        cols=voc_cols,
        pattern=pattern_other_vals
    )

    # Impp values extraction
    impp_vals = extract_values_from_column(
        table=table,
        cols=impp_cols,
        pattern=pattern_other_vals
    )

    # Vmpp values extraction
    vmpp_vals = extract_values_from_column(
        table=table,
        cols=vmpp_cols,
        pattern=pattern_other_vals
    )

    # Pmpp values extraction
    pmpp_vals = extract_values_from_column(
        table=table,
        cols=pmpp_cols,
        pattern=pattern_other_vals
    )

    # FF values extraction
    ff_vals = extract_values_from_column(
        table=table,
        cols=ff_cols,
        pattern=pattern_other_vals
    )

    print(eff_vals)
    print(isc_vals)
    print(voc_vals)
    print(impp_vals)
    print(vmpp_vals)
    print(pmpp_vals)
    print(ff_vals)

    print(len(eff_vals))
    print(len(isc_vals))
    print(len(voc_vals))
    print(len(impp_vals))
    print(len(vmpp_vals))
    print(len(pmpp_vals))
    print(len(ff_vals))



In [259]:
get_electrical(
    table=table,
    axis='vertical'
    )

Number of Columns: 7
['22.400', '22.300', '22.200', '22.100', '22.000', '21.900', '21.800', '21.700', '21.600', '21.500', '21.400']
['9.970', '9.960', '9.950', '9.940', '9.930', '9.920', '9.910', '9.900', '9.890', '9.880', '9.870']
['0.689', '0.688', '0.687', '0.686', '0.685', '0.684', '0.683', '0.682', '0.681', '0.680', '0.679']
['9.550', '9.520', '9.500', '9.470', '9.440', '9.420', '9.390', '9.360', '9.340', '9.310', '9.280']
['0.591', '0.590', '0.589', '0.588', '0.587', '0.586', '0.585', '0.584', '0.583', '0.582', '0.581']
['5.640', '5.620', '5.590', '5.570', '5.540', '5.520', '5.490', '5.470', '5.440', '5.420', '5.390']
[]
11
11
11
11
11
11
0


/usr/local/anaconda3/envs/Thesis/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


In [245]:
patt = "\d+.\d+\D|\d+\D|\d+"

value = "123"

result = re.search(
                pattern=patt,
                string=str(value)
            )

print(value)
print(result.group(0))

123
123


In [5]:
with open("patterns.yaml", "r") as stream:
    try:
        patterns = yaml.safe_load(stream)
    except yaml.YAMLError as e:
        print(e)

In [3]:
sample_ds = Datasheet(
    path_to_excel="/Users/moeezmalik/Documents/Main/work/fraunhofer/thesis/Lightning-Table/validation/tests/files/three.xlsx",
    path_to_clf="nb_classifier.pickle",
    path_to_vec="vectoriser.pickle"
    )

sample_ds.extract_electrical_props(patterns=patterns.get("electrical"))
print(sample_ds.extracted_elec)

Table Axis: vertical
{'eff': {'cols': [1], 'vals': ['22.80%', '22.70%', '22.60%', '22.50%', '22.40%', '22.30%', '22.20%', '22.10%', '22.00%', '21.90%', '21.80%']}, 'ff': {'cols': None, 'vals': None}, 'impp': {'cols': [4], 'vals': ['10.550', '10.544', '10.529', '10.515', '10.502', '10.488', '10.485', '10.472', '10.464', '10.451', '10.437']}, 'isc': {'cols': [6], 'vals': [11.124, '11.119', '11.103', '11.089', '11.077', '11.060', '11.046', '11.033', '11.025', '11.008', '10.989']}, 'pmpp': {'cols': [2], 'vals': ['6.25', '6.22', '6.20', '6.17', '6.14', '6.11', '6.09', '6.06', '6.03', '6.00', '5.98']}, 'vmpp': {'cols': [3], 'vals': ['0.5925', '0.5902', '0.5885', '0.5866', '0.5848', '0.5829', '0.5805', '0.5786', '0.5764', '0.5745', '0.5726']}, 'voc': {'cols': [5], 'vals': ['0.6803', '0.6784', '0.677', '0.6761', '0.6746', '0.6732', '0.6699', '0.6683', '0.6662', '0.6641', '0.6619']}}


/Users/moeezmalik/Documents/Main/work/fraunhofer/thesis/Lightning-Table/validation/cellstructure.py:656: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  na=False


In [5]:
sample_ds.extract_temp_props(patterns=patterns.get("temperature"))

print()

for entity, info in sample_ds.extracted_temp.items():

    print(entity)

    print("row: " + str(info.get("rows")))
    print("vals: " + str(info.get("vals")))

    print()


isc
row: [2]
vals: ['0.07']

pmpp
row: [3]
vals: []

voc
row: [1]
vals: ['-0.36']



/Users/moeezmalik/Documents/Main/work/fraunhofer/thesis/Lightning-Table/validation/cellstructure.py:805: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  flags=re.IGNORECASE,


In [25]:
maybe_horizontal = Datasheet(
    path_to_excel="/Volumes/T7/thesis-data/test/single_excel/c7a6b36f88570815a39e60c07b3db8d31.xlsx",
    path_to_clf="nb_classifier.pickle",
    path_to_vec="vectoriser.pickle"
    )

maybe_horizontal.extract_electrical_props(patterns=patterns.get("electrical"))
print(maybe_horizontal.extracted_elec)

impp_vals = maybe_horizontal.extracted_elec.get("impp").get("vals")

print()
print(impp_vals)

Table Axis: horizontal
{'eff': {'rows': [2], 'vals': ['22.60', '24.5', '25.1']}, 'ff': {'rows': None, 'vals': None}, 'impp': {'rows': [5], 'vals': ['9.882', '10.21', '10.27']}, 'isc': {'rows': [7], 'vals': ['10.870', '11.231', '11.297']}, 'pmpp': {'rows': [3], 'vals': ['5.840', '6.33', '6.47']}, 'vmpp': {'rows': [4], 'vals': ['0.591', '0.62', '0.63']}, 'voc': {'rows': [6], 'vals': ['0.697', '0.732', '0.743']}}

['9.882', '10.21', '10.27']


In [9]:
evaluation = Datasheet(
    path_to_excel="../../Evaluation-Data/fullpipeline/excels/camelot/5b1de46c5106d.xlsx",
    path_to_clf="../../Evaluation-Data/fullpipeline/models/nb_classifier.pickle",
    path_to_vec="../../Evaluation-Data/fullpipeline/models/vectoriser.pickle"
)

evaluation.extract_electrical_props(patterns=patterns.get("electrical"))
elec_props = evaluation.extracted_elec
print(elec_props)

print()

evaluation.extract_temp_props(patterns=patterns.get("temperature"))
print(evaluation.extracted_temp)

Table Axis: horizontal
{'eff': {'rows': None, 'vals': None}, 'ff': {'rows': None, 'vals': None}, 'impp': {'rows': [3], 'vals': ['8.50', '8.72', '8.93']}, 'isc': {'rows': [5], 'vals': ['9.07', '9.18', '9.29']}, 'pmpp': {'rows': [1], 'vals': ['280', '290', '300']}, 'vmpp': {'rows': [2], 'vals': ['32.94', '33.32', '33.72']}, 'voc': {'rows': [4], 'vals': ['39.06', '39.46', '39.87']}}

{'isc': {'rows': [5], 'vals': ['0.036%/K']}, 'pmpp': {'rows': [3], 'vals': ['0.39%/K']}, 'voc': {'rows': [4], 'vals': ['0.34%/K']}}


/Users/moeezmalik/Documents/Main/work/fraunhofer/thesis/Lightning-Table/validation/cellstructure.py:816: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  na=False


In [12]:
with open('yml_test.yml', 'w') as outfile:
    yaml.dump(elec_props, outfile, default_flow_style=False)

In [16]:
gt = [1, 2, 3, 4]

x1 = [1, 2, 3, 4]
x2 = [1, 2, 3, 4, 5]
x3 = [1, 2, 3]
x4 = [0, 1, 2, 3, 4, 5]
x5 = []
x6 = None

def confusion_matrix(true, preds):

    if true is None:
        true = []

    if preds is None:
        preds = []

    intersection_count = len(set(preds) & set(true))

    tp = intersection_count
    fp = len(preds) - intersection_count
    fn = len(true) - intersection_count

    return tp, fp, fn

print(confusion_matrix(gt, x1))
print(confusion_matrix(gt, x2))
print(confusion_matrix(gt, x3))
print(confusion_matrix(gt, x4))
print(confusion_matrix(gt, x5))
print(confusion_matrix(gt, x6))

(4, 0, 0)
(4, 1, 0)
(3, 0, 1)
(4, 2, 0)
(0, 0, 4)
(0, 0, 4)
